In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
print("Setup complete")

In [ ]:
iris = load_iris()
X = StandardScaler().fit_transform(iris.data)
y = iris.target
print(f"Data ready: {X.shape}")

In [ ]:
# SGD with different learning rates
learning_rates = [0.001, 0.01, 0.1, 1.0]
scores_dict = {}

for lr in learning_rates:
    sgd = SGDClassifier(learning_rate='constant', eta0=lr, random_state=42, max_iter=1000)
    scores = cross_val_score(sgd, X, y, cv=5)
    scores_dict[f'LR={lr}'] = scores.mean()
    print(f"Learning rate {lr}: {scores.mean():.3f}")

In [ ]:
# Convergence plot
sgd = SGDClassifier(learning_rate='constant', eta0=0.01, random_state=42, warm_start=True)
train_scores = []
for epoch in range(1, 100, 5):
    sgd.max_iter = epoch
    sgd.fit(X[:100], y[:100])
    score = sgd.score(X[:100], y[:100])
    train_scores.append(score)

plt.figure(figsize=(10, 5))
plt.plot(range(1, 100, 5), train_scores, 'o-')
plt.xlabel('Epochs')
plt.ylabel('Training Accuracy')
plt.title('SGD Convergence')
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# Regularization
alphas = [0.0001, 0.001, 0.01, 0.1, 1.0]
reg_scores = []
for alpha in alphas:
    sgd = SGDClassifier(alpha=alpha, random_state=42, max_iter=1000)
    score = cross_val_score(sgd, X, y, cv=5).mean()
    reg_scores.append(score)

plt.figure(figsize=(10, 5))
plt.plot(alphas, reg_scores, 'o-')
plt.xlabel('Alpha (Regularization)')
plt.ylabel('CV Score')
plt.xscale('log')
plt.title('Regularization Effect')
plt.grid(alpha=0.3)
plt.show()

In [ ]:
# Loss functions
losses = ['hinge', 'log', 'modified_huber']
loss_scores = {}
for loss in losses:
    sgd = SGDClassifier(loss=loss, random_state=42, max_iter=1000)
    score = cross_val_score(sgd, X, y, cv=5).mean()
    loss_scores[loss] = score
    print(f"Loss {loss}: {score:.3f}")

In [ ]:
# Batch vs Online
sgd_batch = SGDClassifier(n_jobs=-1, random_state=42, max_iter=1000)
batch_score = cross_val_score(sgd_batch, X, y, cv=5).mean()
print(f"SGD Score: {batch_score:.3f}")

In [ ]:
# Comparison
plt.figure(figsize=(10, 5))
plt.bar(scores_dict.keys(), scores_dict.values())
plt.ylabel('CV Score')
plt.title('Learning Rate Impact')
plt.xticks(rotation=45)
plt.ylim([0, 1])
plt.grid(alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

In [ ]:
test_results = []
test1 = max(scores_dict.values()) > 0.9
test_results.append(("Test 1: Best LR > 0.9", test1))
test2 = len(train_scores) > 0 and max(train_scores) > 0.7
test_results.append(("Test 2: Convergence works", test2))
test3 = len(reg_scores) == 5
test_results.append(("Test 3: Regularization tested", test3))
test4 = len(loss_scores) == 3
test_results.append(("Test 4: Loss functions tested", test4))
test5 = batch_score > 0.9
test_results.append(("Test 5: SGD accuracy good", test5))
passed = sum(1 for _, r in test_results if r)
print(f"\nPASSED: {passed}/{len(test_results)}")